In [1]:
%matplotlib inline
import os
import time
import requests
import urllib3
from tqdm import tqdm
from lxml import etree
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix
import numpy as np
import seaborn as sn
import warnings
import itertools
warnings.filterwarnings('ignore')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
year = "20162017"
for i in tqdm(range(1, 99)):
    for j in range(1, 99):
        file = "h-tk-%s-%s-%s" %(year, str(i), str(j))
        url = "https://zoek.officielebekendmakingen.nl/%s" %file
        resp = requests.get(url + ".xml", verify=False, allow_redirects=False)
        if resp.status_code == 200:
            hand_file = open('HandelingenTK/%s.xml' %file, 'wb')
            hand_file.write(resp.content)
            hand_file.close()
            resp = requests.get(url+"/metadata.xml", verify=False, \
                                                allow_redirects=False)
            meta_file = open('HandelingenTKmeta/meta%s.xml' %file, 'wb')
            meta_file.write(resp.content)
            meta_file.close()    
        else:
            break

  0%|                                                                                           | 0/98 [00:00<?, ?it/s]